In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

#### Read data

In [31]:
data = pd.read_csv('C:/Users/lisab/OneDrive/Desktop/DA_labs/Week_7/Lesson/learningSet.csv')

C:\Users\lisab\AppData\Local\Temp\ipykernel_23168\911819598.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:/Users/lisab/OneDrive/Desktop/DA_labs/Week_7/Lesson/learningSet.csv')


In [58]:
print(data.shape)
#data.head(10)

(95412, 481)


#### Get all categorical columns

In [5]:
categorical = data.select_dtypes(object)
categorical.head()

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


#### Check for null values

In [10]:
pd.set_option('display.max_rows', None)
categorical.isna().sum()*100/len(categorical)

OSOURCE     0.000000
STATE       0.000000
ZIP         0.000000
MAILCODE    0.000000
PVASTATE    0.000000
NOEXCH      0.000000
RECINHSE    0.000000
RECP3       0.000000
RECPGVG     0.000000
RECSWEEP    0.000000
MDMAUD      0.000000
DOMAIN      0.000000
CLUSTER     0.000000
AGEFLAG     0.000000
HOMEOWNR    0.000000
CHILD03     0.000000
CHILD07     0.000000
CHILD12     0.000000
CHILD18     0.000000
GENDER      0.000000
DATASRCE    0.000000
SOLP3       0.000000
SOLIH       0.000000
MAJOR       0.000000
GEOCODE     0.000000
COLLECT1    0.000000
VETERANS    0.000000
BIBLE       0.000000
CATLG       0.000000
HOMEE       0.000000
PETS        0.000000
CDPLAY      0.000000
STEREO      0.000000
PCOWNERS    0.000000
PHOTO       0.000000
CRAFTS      0.000000
FISHER      0.000000
GARDENIN    0.000000
BOATS       0.000000
WALKER      0.000000
KIDSTUFF    0.000000
CARDS       0.000000
PLATES      0.000000
LIFESRC     0.000000
PEPSTRFL    0.000000
RFA_2       0.000000
RFA_3       0.000000
RFA_4       0

##### Just the column GEOCODE2 has null values, but is under the treshold of 85% and will be kept. In the next step all columns will be checked for " " values and replaced by NaN.

In [33]:
categorical['MAILCODE'].value_counts()

     94013
B     1399
Name: MAILCODE, dtype: int64

In [34]:
categorical['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [35]:
categorical['MAILCODE'] = categorical['MAILCODE'].apply(lambda x: x.replace(" ", "A"))

In [37]:
#categorical.head(10)

#### Replace all " " values with NaN

In [38]:
categorical = categorical.apply(lambda x: x.replace(" ", np.NaN))

In [39]:
categorical.head(10)

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,GENDER,DATASRCE,SOLP3,SOLIH,MAJOR,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,T2,36,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,S1,14,E,H,NaN,NaN,NaN,M,M,3,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L2G,A2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,A1E,NaN,NaN,NaN,L1E,NaN,NaN,N1E,N1E,N1E,N1E,NaN,F1E,L,G,X,X,X,A
2,AMH,NC,27017,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,R2,43,NaN,U,NaN,NaN,NaN,NaN,M,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4F,S4F,S4F,NaN,S4F,S4F,S4F,S4F,S4F,S4F,NaN,S4D,S4D,NaN,NaN,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,R2,44,E,U,NaN,NaN,NaN,NaN,F,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,NaN,S4E,S4E,S4E,S4E,S4E,S4E,S2D,S2D,A1D,A1D,A1D,A1D,NaN,NaN,L,E,X,X,X,C
4,NaN,FL,33176,A,NaN,0,X,X,NaN,NaN,XXXX,S2,16,E,H,NaN,NaN,NaN,NaN,F,3,NaN,12,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,NaN,NaN,Y,NaN,Y,NaN,Y,NaN,Y,NaN,3,NaN,L2F,A2F,A2F,A2F,A1D,I2D,A1E,A1E,L1D,A1E,A1E,L1D,L3D,NaN,L3D,A2D,A2D,A3D,A3D,A3D,I4E,A3D,A3D,L,F,X,X,X,A
5,CWR,AL,35603,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,T2,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L1F,A1F,A1F,A1F,A1F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,N3F,N2E,N2E,N2E,N2E,N2E,N2E,NaN,F1E,L,F,X,X,X,C
6,DRK,IN,46755,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,T2,40,E,H,NaN,NaN,F,NaN,F,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,X,L1E,A1E,A1E,NaN,A1E,S2E,S2E,S3E,S3E,S3E,S3E,S3E,S2E,NaN,S3E,S2E,S2E,A2E,NaN,A2E,A2E,A3E,A3E,L,E,X,X,X,D
7,NWN,LA,70611,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,T2,39,NaN,U,NaN,NaN,NaN,NaN,F,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L3E,A3E,A3E,A3E,A3E,A3E,A3E,A3E,A2D,A2D,A2D,A2D,A1D,NaN,N2D,N2D,N2D,N2D,N2D,N2D,N2D,NaN,F1D,L,E,X,X,X,C
8,LIS,IA,51033,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,R2,45,NaN,U,NaN,NaN,NaN,NaN,M,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L1F,A1F,A1F,A1F,A1F,S2F,S2F,S2F,A1F,A1F,A1F,A1F,A1F,NaN,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,L,F,X,X,X,D
9,MSD,TN,37127-,A,NaN,0,NaN,NaN,NaN,NaN,XXXX,T1,35,I,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L1F,A1F,A1F,NaN,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,NaN,NaN,A1E,A1E,A1E,N2E,N2E,N2E,N2E,NaN,NaN,L,F,X,X,X,B


#### Get a DataFrame with the percentage of NaN values for each column

In [42]:
nulls_percent_df = pd.DataFrame(categorical.isna().sum()*100/len(categorical)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']

In [57]:
#nulls_percent_df

#### Get just the column with have a percentage of NaN values above 85%

In [44]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>85]
columns_above_threshold['column_name']

4     PVASTATE
6     RECINHSE
7        RECP3
8      RECPGVG
9     RECSWEEP
15     CHILD03
16     CHILD07
17     CHILD12
18     CHILD18
21       SOLP3
22       SOLIH
23       MAJOR
25    COLLECT1
26    VETERANS
27       BIBLE
28       CATLG
29       HOMEE
31      CDPLAY
32      STEREO
33    PCOWNERS
34       PHOTO
35      CRAFTS
36      FISHER
37    GARDENIN
38       BOATS
39      WALKER
40    KIDSTUFF
41       CARDS
42      PLATES
Name: column_name, dtype: object

#### Get a list with a columns to be dropped

In [45]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

['PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'SOLP3', 'SOLIH', 'MAJOR', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES']


#### Append columns OSOURCE and ZIP, because they should also be dropped although being under the treshold for NaN values

In [46]:
drop_columns_list.append('OSOURCE')
drop_columns_list.append('ZIP')

In [47]:
print(drop_columns_list)

['PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18', 'SOLP3', 'SOLIH', 'MAJOR', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'OSOURCE', 'ZIP']


#### Drop all columns in drop_columns_list from the categorical dataset

In [48]:
categorical = categorical.drop(drop_columns_list, axis=1)

In [52]:
categorical.head(10)

,STATE,MAILCODE,NOEXCH,MDMAUD,DOMAIN,CLUSTER,AGEFLAG,HOMEOWNR,GENDER,DATASRCE,GEOCODE,PETS,LIFESRC,PEPSTRFL,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,IL,A,0,XXXX,T2,36,NaN,NaN,F,NaN,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,CA,A,0,XXXX,S1,14,E,H,M,3,2,NaN,NaN,NaN,L2G,A2G,A2G,A2G,A2G,A1E,A1E,A1E,A1E,A1E,A1E,NaN,NaN,NaN,L1E,NaN,NaN,N1E,N1E,N1E,N1E,NaN,F1E,L,G,X,X,X,A
2,NC,A,0,XXXX,R2,43,NaN,U,M,3,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4F,S4F,S4F,NaN,S4F,S4F,S4F,S4F,S4F,S4F,NaN,S4D,S4D,NaN,NaN,S4D,S4D,S3D,L,E,X,X,X,C
3,CA,A,0,XXXX,R2,44,E,U,F,3,NaN,NaN,NaN,X,L4E,S4E,S4E,S4E,S4E,S4E,S4E,S4E,NaN,S4E,S4E,S4E,S4E,S4E,S4E,S2D,S2D,A1D,A1D,A1D,A1D,NaN,NaN,L,E,X,X,X,C
4,FL,A,0,XXXX,S2,16,E,H,F,3,NaN,NaN,3,NaN,L2F,A2F,A2F,A2F,A1D,I2D,A1E,A1E,L1D,A1E,A1E,L1D,L3D,NaN,L3D,A2D,A2D,A3D,A3D,A3D,I4E,A3D,A3D,L,F,X,X,X,A
5,AL,A,0,XXXX,T2,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,L1F,A1F,A1F,A1F,A1F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,A2F,N3F,N2E,N2E,N2E,N2E,N2E,N2E,NaN,F1E,L,F,X,X,X,C
6,IN,A,0,XXXX,T2,40,E,H,F,3,NaN,Y,3,X,L1E,A1E,A1E,NaN,A1E,S2E,S2E,S3E,S3E,S3E,S3E,S3E,S2E,NaN,S3E,S2E,S2E,A2E,NaN,A2E,A2E,A3E,A3E,L,E,X,X,X,D
7,LA,A,0,XXXX,T2,39,NaN,U,F,1,NaN,NaN,NaN,NaN,L3E,A3E,A3E,A3E,A3E,A3E,A3E,A3E,A2D,A2D,A2D,A2D,A1D,NaN,N2D,N2D,N2D,N2D,N2D,N2D,N2D,NaN,F1D,L,E,X,X,X,C
8,IA,A,0,XXXX,R2,45,NaN,U,M,3,NaN,NaN,NaN,X,L1F,A1F,A1F,A1F,A1F,S2F,S2F,S2F,A1F,A1F,A1F,A1F,A1F,NaN,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,L,F,X,X,X,D
9,TN,A,0,XXXX,T1,35,I,NaN,M,NaN,3,NaN,NaN,NaN,L1F,A1F,A1F,NaN,A1F,A1F,A1F,A1F,A1F,A1F,A1F,A1F,NaN,NaN,A1E,A1E,A1E,N2E,N2E,N2E,N2E,NaN,NaN,L,F,X,X,X,B


In [50]:
categorical.shape

(95412, 43)

#### Working on gender column

##### Replace NaN values first

In [53]:
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')

F    51277
M    39094
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64


##### Reducing number of categories to 'M', 'F' and 'other'

In [54]:
df = pd.DataFrame(categorical['GENDER'].value_counts()).reset_index()

df.columns = ['gender', 'count']
other_genders = list(df[df['count']<2500]['gender'])
def clean_gender(x):
    if x in other_genders:
        return 'other'
    else:
        return x
    
categorical['GENDER'] = list(map(clean_gender, categorical['GENDER']))

In [55]:
print(categorical['GENDER'].value_counts())

F        54234
M        39094
other     2084
Name: GENDER, dtype: int64
